In [57]:
import pandas as pd

In [58]:
old_data = pd.read_excel("references/image_info.xlsx")
new_data = pd.read_excel("references/NEW_Image_info.xlsx")

MIGHT BE CHANGED

In [59]:
import pandas as pd
import numpy as np

# Ensure reproducibility
np.random.seed(42)

# Filter rows where Research Group is 'MCI'
mci_rows = old_data[old_data['Research Group'] == 'MCI']

# Sample 32 random rows from those filtered rows
rows_to_remove = mci_rows.sample(n=41)

# Drop these rows from the original DataFrame
old_data = old_data.drop(rows_to_remove.index)

# Verify changes by displaying the shape of old_data and counts of Research Group
print(old_data.shape)
print(old_data['Research Group'].value_counts())


(315, 29)
EMCI    108
CN      104
LMCI     56
SMC      24
AD       22
MCI       1
Name: Research Group, dtype: int64


In [60]:
# Rename the 'Subject ID' column to 'Subject' in the new_data DataFrame
new_data.rename(columns={'Subject ID': 'Subject'}, inplace=True)

# Verify the renaming by checking the columns of new

In [61]:
subject_columns = ['Subject', 'Research Group', 'Age']

In [62]:
import pandas as pd

# Define the columns you want to keep
subject_columns = ['Subject', 'Research Group', 'Age', 'Sex']

# Select these columns from both dataframes
selected_old_data = old_data[subject_columns]
selected_new_data = new_data[subject_columns]

# Concatenate the two dataframes
df = pd.concat([selected_old_data, selected_new_data], ignore_index=True)

# Remove duplicates to ensure each row is unique
df = df.drop_duplicates()

# Optionally, you can sort the dataframe based on one or more columns
df.sort_values(by=['Subject', 'Age'], inplace=True)

# Display the first few rows to verify
print(df.head())


      Subject Research Group   Age Sex
0  002_S_0295             CN  90.0   M
2  002_S_0413             CN  81.5   F
4  002_S_0685             CN  95.8   F
6  002_S_1261             CN  75.3   F
8  002_S_1280             CN  75.1   F


In [63]:
df

,Subject,Research Group,Age,Sex
0,002_S_0295,CN,90.0,M
2,002_S_0413,CN,81.5,F
4,002_S_0685,CN,95.8,F
6,002_S_1261,CN,75.3,F
8,002_S_1280,CN,75.1,F
...,...,...,...,...
748,941_S_6052,MCI,88.2,F
749,941_S_6068,MCI,75.8,M
750,941_S_6345,MCI,79.3,M
751,941_S_6803,MCI,75.1,F


In [64]:
# Replace 'LMCI' with 'MCI' in the 'Research Group' column
df['Research Group'] = df['Research Group'].replace('LMCI', 'MCI')

# Verify the change by displaying the counts of each Research Group again
print(df['Research Group'].value_counts())


CN      280
MCI     280
EMCI     73
SMC      23
AD       21
Name: Research Group, dtype: int64


In [65]:
# Filter the DataFrame to include only rows where 'Research Group' is 'MCI' or 'CN'
filtered_df = df[df['Research Group'].isin(['MCI', 'CN'])]

# Display the first few rows to verify the filtering
print(filtered_df.head())

# Optionally, check the counts of each Research Group to confirm the filter
print(filtered_df['Research Group'].value_counts())


      Subject Research Group   Age Sex
0  002_S_0295             CN  90.0   M
2  002_S_0413             CN  81.5   F
4  002_S_0685             CN  95.8   F
6  002_S_1261             CN  75.3   F
8  002_S_1280             CN  75.1   F
CN     280
MCI    280
Name: Research Group, dtype: int64


In [67]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 560 entries, 0 to 752
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Subject         560 non-null    object 
 1   Research Group  560 non-null    object 
 2   Age             560 non-null    float64
 3   Sex             560 non-null    object 
dtypes: float64(1), object(3)
memory usage: 21.9+ KB


In [69]:
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit

# Reset the index of filtered_df to ensure continuity and proper alignment
filtered_df.reset_index(drop=True, inplace=True)

# Initialize the StratifiedShuffleSplit object for the first split (train_val/test)
splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

# Create a new column for the dataset split tags
filtered_df['dataset_split'] = None

# First, split into test and the combined train/validation sets
for train_val_idx, test_idx in splitter.split(filtered_df, filtered_df['Research Group']):
    filtered_df.loc[test_idx, 'dataset_split'] = 'test'
    train_val_set = filtered_df.loc[train_val_idx]

# Now, split the train_val_set into training and validation
# Note: To achieve 70% train and 10% validation of the total dataset, we set validation to be 1/8 (12.5%) of the 80% train_val_set
splitter = StratifiedShuffleSplit(n_splits=1, test_size=1/8, random_state=42)  # 0.125 x 0.8 = 0.1 (10%)
for train_idx, val_idx in splitter.split(train_val_set, train_val_set['Research Group']):
    filtered_df.loc[train_val_set.index[train_idx], 'dataset_split'] = 'train'
    filtered_df.loc[train_val_set.index[val_idx], 'dataset_split'] = 'validation'

# Check the distribution of each dataset split within each research group
print(filtered_df['dataset_split'].value_counts())
print(filtered_df.groupby(['Research Group', 'dataset_split']).size())


train         392
test          112
validation     56
Name: dataset_split, dtype: int64
Research Group  dataset_split
CN              test              56
                train            196
                validation        28
MCI             test              56
                train            196
                validation        28
dtype: int64


C:\Users\Micha\AppData\Local\Temp\ipykernel_93532\3775637731.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['dataset_split'] = None


In [70]:
import pandas as pd

# Grouping by Research Group and Sex, and then aggregating
descriptive_stats = filtered_df.groupby(['Research Group', 'Sex']).agg(
    Number_of_Subjects=pd.NamedAgg(column='Subject', aggfunc='count'),
    Average_Age=pd.NamedAgg(column='Age', aggfunc='mean'),
    Age_Std_Dev=pd.NamedAgg(column='Age', aggfunc='std')
)

# Formatting the Average Age and Standard Deviation
descriptive_stats['Average_Age'] = descriptive_stats['Average_Age'].round(1)
descriptive_stats['Age_Std_Dev'] = descriptive_stats['Age_Std_Dev'].round(1)
descriptive_stats['Average_Age_with_Std'] = descriptive_stats['Average_Age'].astype(str) + ' ± ' + descriptive_stats['Age_Std_Dev'].astype(str)

# Display the descriptive statistics table
print(descriptive_stats[['Number_of_Subjects', 'Average_Age_with_Std']])


                    Number_of_Subjects Average_Age_with_Std
Research Group Sex                                         
CN             F                   138           75.9 ± 6.7
               M                   142           77.5 ± 6.3
MCI            F                   112           73.5 ± 8.4
               M                   168           75.6 ± 7.5


In [72]:
import pandas as pd

# Grouping by Research Group and dataset_split, and then aggregating
group_split_stats = filtered_df.groupby(['Research Group', 'dataset_split']).agg(
    Number_of_Subjects=pd.NamedAgg(column='Subject', aggfunc='count'),
    Average_Age=pd.NamedAgg(column='Age', aggfunc='mean'),
    Age_Std_Dev=pd.NamedAgg(column='Age', aggfunc='std')
)

# Formatting the Average Age and Standard Deviation
group_split_stats['Average_Age'] = group_split_stats['Average_Age'].round(1)
group_split_stats['Age_Std_Dev'] = group_split_stats['Age_Std_Dev'].round(1)
group_split_stats['Average_Age_with_Std'] = group_split_stats['Average_Age'].astype(str) + ' ± ' + group_split_stats['Age_Std_Dev'].astype(str)

# Display the descriptive statistics table
print(group_split_stats[['Number_of_Subjects', 'Average_Age_with_Std']])


                              Number_of_Subjects Average_Age_with_Std
Research Group dataset_split                                         
CN             test                           56           75.9 ± 6.4
               train                         196           76.9 ± 6.4
               validation                     28           77.0 ± 7.6
MCI            test                           56           75.5 ± 8.0
               train                         196           74.5 ± 8.0
               validation                     28           75.1 ± 7.6
